# A

In [1]:
k = int(input())
print('ACL' * k)

3
ACLACLACL


# B

In [7]:
a,b,c,d = map(lambda x: int(x), input().split())

if (b < c) | (d < a):
    print('No')
else:
    print('Yes')

10 30 20 40
Yes


# C

In [29]:
import math

class UnionFind:
    def __init__(self, size):
        self.parent_indexes = [-1] * size
        
    def find(self, n):
        if self.parent_indexes[n] < 0:
            return n
        # 探しているnodeが指し示しているのが現在の親ではない場合、
        # 現在の親に書き換え(メモ化) した上で返す
        self.parent_indexes[n] = self.find(self.parent_indexes[n])
        return self.parent_indexes[n] 
    
    def union(self, n1, n2):
        if self.same_group(n1, n2):
            return False
        
        # サイズの大きい方の集団の親が親であり続ける
        if self.size(n1) >= self.size(n2):
            tmp = self.size(n2)
            self.parent_indexes[self.find(n2)] = self.find(n1)
            self.parent_indexes[self.find(n1)] -= tmp
            return True
        
        # サイズが小さい方の集団の親は親を引退する
        tmp = self.size(n1)
        self.parent_indexes[self.find(n1)] = self.find(n2)
        self.parent_indexes[self.find(n2)] -= tmp
        return True
        
    # その要素が所属する集団の要素数
    def size(self, n):
        return -self.parent_indexes[self.find(n)]
    
    def same_group(self, n1, n2):
        return self.find(n1) == self.find(n2)

n, m = map(lambda x: int(x), input().split())


uf = UnionFind(n)

for _ in range(m):
    a, b = map(lambda x: int(x), input().split())
    uf.union(a-1, b-1)
    
num_parents = 0
    
for p_index in uf.parent_indexes:
    if p_index < 0:
        num_parents += 1
# print(uf.parent_indexes)
print(num_parents - 1)

3 1
1 2
1


# D

In [9]:
%load_ext cythonmagic

The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


In [11]:
%%timeit
%%cython

In [16]:
import sys
sys.setrecursionlimit(3000000) 

n, k = map(lambda x: int(x), input().split())

list_a = []
for _ in range(n):
     list_a.append(int(input()))
        
memo = dict()

# i番目以降の要素からなるsublistの長さとして最長のものの長さを返す
def sublist_max_length(current_value, i):
    if (current_value, i) in memo.keys():
        return memo[(current_value, i)]
    if i == n-1:
        if (current_value is None) or (abs(current_value - list_a[i]) <= k):
            memo[(current_value, i)] = 1
            return memo[(current_value, i)]
        memo[(current_value, i)] = 0
        return memo[(current_value, i)]
    
    if (current_value is None) or (abs(current_value - list_a[i]) <= k):
        # iの値をsublistに含めるor含めない
        memo[(current_value, i)] = max(1+sublist_max_length(list_a[i], i+1), sublist_max_length(current_value, i+1))
        return memo[(current_value, i)]
    memo[(current_value, i)] = sublist_max_length(current_value, i+1)
    return memo[(current_value, i)]

print(sublist_max_length(None, 0))

433 µs ± 9.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
## random testの作成

import sys
sys.setrecursionlimit(3000000)

import random
memo = dict()
n = random.randint(1, 30000)
k = random.randint(0, 30000)
list_a = [random.randint(0, 30000) for _ in range(n)]

print(sublist_max_length(None, 0))

3


## なぜかできなかったと思ったら、再帰のスタックオーバーフローだった

以下が必要だった。
```
import sys
sys.setrecursionlimit(適当に大きな数)
```

入れてもTLEだが。


## 解法①動的計画法DP

In [36]:
n, k = map(lambda x: int(x), input().split())

list_a = []
max_a = 0
for _ in range(n):
    a = int(input())
    list_a.append(a)
    max_a = max(max_a, a)

# n回操作後のdp配列のm番目(m=0,1,2...)の要素は、
# 「前からn番目までの数列について、最後の数字がm+1となるような部分列として考えられるものの中で最長のものの長さ」
# とする。
# 
dp = [0] * max_a

for a in list_a:
    i_new_number = a - 1
    updated_length = 0
    for i in range(max(0, i_new_number-k), min(max_a, i_new_number+k+1)):
        updated_length = max(updated_length, dp[i]+1)
    dp[i_new_number] = updated_length
#     print(dp)
print(max(dp))

10 3
1
5
4
3
8
6
9
7
2
4
7


合ってるけど、まだTLEなので、`updated_length = max(dp[i-k]~dp[i+k])`の部分は区間の処理なのでsegment treeを使う。

- 競プロじゃなくて理論だと、平衡二分探索木(SelfBalancedSearch)を使えば、もっと色々な操作ができるし、計算量は同じ。
- でもsegment treeだと、定数倍(???)がめっちゃ速いし、実装量も少ない


今回の問題だと、以下をしたい。のでまさにSegmentTreeを使う

- 一点更新(代入)
- 区間のmaxを取得

## 解法②区間木SegmentTree
